In [1]:
import numpy as np
import glob as glob
import SimpleITK as sitk
from tqdm import tqdm
import cv2
import pydicom
import os
import matplotlib.pyplot as plt
from PIL import Image
from tabulate import tabulate
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure, feature

from scipy.spatial import distance
import skimage

import glob as glob
import SimpleITK as sitk
from tqdm import tqdm
import cv2
import pydicom
import os
import matplotlib.pyplot as plt
from tabulate import tabulate
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure, feature
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage import color
from skimage import io
import cv2
import math
from matplotlib import image
from matplotlib import pyplot
from scipy import ndimage

import numpy as np
import ipywidgets as ipyw
import matplotlib.pyplot as plt
import pydicom
from pydicom.data import get_testdata_files
from pydicom.filereader import read_dicomdir

ModuleNotFoundError: No module named 'tabulate'

In [2]:
!pip install tabulate

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\Scripts\pip-script.py", line 10, in <module>
    sys.exit(main())
TypeError: 'module' object is not callable


In [ ]:
class dicom:

    def __init__(self):
        self.read=read()
        self.modify=modify()
        self.display=display()
        self.process=process()

In [ ]:
class read:
  
    def __init__(self):#, folder_path):


        pass
        #super().__init__()

    def scan(self,folder_path): ##Public func
        '''
        This function takes path of scan as input and returns scan array in 4D and its metadeta in a list. 
        The 4D array contains multiple 3D arrays corresponding to each series of the scan.
        The meta data will be returned in a list corresponding to each slice in the scan.

        
        Parameters:
                scan_path : path of given dicom scan

        Returns:
                (float): Scan (numpy array)
                        : Meta Data (list)
        '''      
       
        
        files = self._sorted_files(folder_path)
        f, myscan, mytags = self._read_scan(folder_path,files)
        return np.array(myscan), mytags

    def _sorted_files(self,folder): ##Private function
        ins=[]
        ori = []
        ser=[]
        f= os.listdir(folder)
        for name in tqdm(f):
            itkimage = sitk.ReadImage(os.path.join(folder,name))
            temp= float(itkimage.GetMetaData('0020|0013')) # instance number 
            temp1= itkimage.GetMetaData('0020|000e') #series instance UID 
            ins.append(int(temp))
            ser.append(temp1)
        series=np.unique(ser)

        files=[[x for sr,_,x in sorted(zip(ser,ins,f)) if sr==s] for s in series]
        return files


    def _read_scan(self,folder_path,files): ##Private func
        axial  =  np.array([1., 0., 0., 0., 1., 0.])
        coronal = np.array([1., 0., 0., 0., 0., -1.])
        sagittal= np.array([0., 1., 0., 0., 0., -1.])
        myscan = []
        mytags = []
        f=[]
        for i in tqdm (range (len(files))):
            img=[]
            file=[]
            tags = []
            for filename in files[i]:
                ds = pydicom.dcmread(os.path.join(folder_path,filename))
                numpyImage = ds.pixel_array
                del ds[0x7fe0, 0x0010]
                img.append(numpyImage)
                file.append(filename)
                tags.append(ds)
            mytags.append(tags)
            myscan.append(np.array(img))
            f.append(file)
        return f, myscan, mytags

    def Slice(self, slice_path): ##Public
        '''
        This function takes path of slice as input and returns slice array and its metadeta.
        
        Parameters:
                slice_path : path of given dicom file

        Returns:
                (int)(float): Slice 
                            : Meta Data
        '''         
        ds = pydicom.dcmread(slice_path)
        numpyImage = ds.pixel_array
        return numpyImage, ds



    def tag(self,tag_id,ds): ##Public
        '''This function takes the tag code and metadata as input and returns the value of provided tag code.
        
        Parameters:
                tag_id : a tag that identifies the attribute, usually in the format (XXXX,XXXX) with hexadecimal numbers
                ds : meta deta of dicom slice

        Returns:
                (int)(float)(String): Tag Value 
                            
        '''
        (x1,x2) = tags
        if (x1,x2) in ds:
            print(ds[x1,x2])
            return ds[x1,x2].value
        else:
            print('Tag not found')
    
    ################################################################
    def getSeriesPlane(self,ds): ##public
        '''This function  
        
        Parameters:
                ds= meta deta
                
        Returns:
                Series Orientation, Series Instance UID                            
        '''
        l=[]
        if ((0x0020,0x0037) in ds):
            o=ds[0x0020,0x0037].value
            
        else:
            o=''
        if ((0x0020,0x000e) in ds):
            s=ds[0x0020,0x000e].value
            #print("hi")
        else:
            s=''
        l.append([o,s])

        return l
#########################################################################
    

    def orientation(self,orientation_tag_value): ##Public func
        '''
       This function returns the orientation of given
        
        Parameters:
                orientation_tag_value : orientation tag

        Returns:
                (string): Orientation of tag [axial, sagittal or coronal] 
                        
        '''
        axial  =  np.array([1., 0., 0., 0., 1., 0.])
        coronal = np.array([1., 0., 0., 0., 0., -1.])
        sagittal= np.array([0., 1., 0., 0., 0., -1.])

        a = np.around(orientation_tag_value)

        if (a==coronal).all():
            return 'coronal'
        elif (a==axial).all():
            return 'axial'
        elif (a==sagittal).all():
            return 'sagittal'
        else:
            print('orientation not found')
            return ''

    def seriesData(self,ds):   ##Public func
        '''This function returns an array od size [n,3] where n is the number of scans and of sopseriesUID, series_number and plane corresponding to each series.
         Parameters:
                ds  :  a list containing scan meta data

        Returns:
                (list):  [sopseriesUID, series_number, view corresponding to each series] 
                        
        '''

        n=len(ds)
        l=[]
        for i in range(n):
            a = ds[i][0][0x0020,0x000e].value if ((0x0020,0x000e) in ds[i][0]) else ''
            if ((0x0020,0x0037) in ds[i][0]):
                b = self.orientation(ds[i][0][0x0020,0x0037].value)
            else:
                b = 'no orientation tag'
            c = ds[i][0][0x0020,0x0011].value if ((0x0020,0x0011) in ds[i][0]) else ''
            l.append(np.array([a,b,c]))

        return np.array(l)




    def getImagePlane(self,ds):  ##Public func
        #Pixel Spacing
        '''it will  returns  all the tags within ImagePlane  module
         Parameters:
                ds  :  scan meta deta

        Returns:
                list: Tag Name, Tag Code and Tag Values with in imagePlane Module 
                        
        '''
        l=[]
        #a = ds[0x0020,0x000e].value
        if ((0x0020,0x0050) in ds):
            a=ds[0x0020,0x0050]
        else:
            a=''
        if ((0x0020,0x0032) in ds):
            a1=ds[0x0020,0x0032]
        else:
            a1=''
        if ((0x0020,0x0037) in ds):
            a2=ds[0x0020,0x0037]
        else:
            a2=''
        if ((0x0020,0x1041) in ds):
            a3=ds[0x0020,0x1041]
        else:
            a3=''
        if ((0x0020,0x0030) in ds):
            a4=ds[0x0020,0x0030]
        else:
            a4=''

       # print(data)
        l.append([a,a1,a2,a3,a4])
        return l



    def getFrameOfReference(self,meta_data): ##Public func
        '''it will  returns  all the tags within FrameOfReference  module
         Parameters:
                ds  :  scan meta deta

         Returns:
                list : Tag Name, Tag Code and Tag Values with in FrameOfReference Module 
                        
         '''
        #data = meta_data.group_dataset(0x0020)
        l=[]
        #a = ds[0x0020,0x000e].value
        if ((0x0020,0x0052) in ds):
            a=ds[0x0020,0x0052]
        else:
            a=''
        if ((0x0020,0x1040) in ds):
            a1=ds[0x0020,0x1040]
        else:
            a1=''

       # print(data)
        l.append([a,a1])
        return l
    
    def getPatientStudy(self,ds):
        '''it will  returns  all the tags within getPatientStudy  module
         Parameters:
                ds  :  scan meta deta

         Returns:
                list : Tag Name, Tag Code and Tag Values with in getPatientStudy Module 
                        
        '''  
        l=[]
        #a = ds[0x0020,0x000e].value
        if ((0x0008,0x1080) in ds):
            a=ds[0x0008,0x1080]
        else:
            a=''
        if ((0x0008,0x1084) in ds):
            a1=ds[0x0008,0x1084]
        else:
            a1=''
        if ((0x0010,0x1010) in ds):
            a2=ds[0x0010,0x1010]
        else:
            a2=''
        if ((0x0010,0x1020) in ds):
            a3=ds[0x0010,0x1020]
        else:
            a3=''
        if ((0x0010,0x1021) in ds):
            a4=ds[0x0010,0x1021]
        else:
            a4=''
        if ((0x0010,0x1022) in ds):
            a5=ds[0x0010,0x1022]
        else:
            a5=''
        if ((0x0010,0x1023) in ds):
            a6=ds[0x0010,0x1023]
        else:
            a6=''
        if ((0x0010,0x1024) in ds):
            a7=ds[0x0010,0x1024]
        else:
            a7=''
        if ((0x0010,0x1030) in ds):
            a8=ds[0x0010,0x1030]
        else:
            a8=''
        if ((0x0010,0x2000) in ds):
            a9=ds[0x0010,0x2000]
        else:
            a9=''
        if ((0x0010,0x2110) in ds):
            a10=ds[0x0010,0x2110]
        else:
            a10=''
        if ((0x0010,0x2180) in ds):
            a11=ds[0x0010,0x2180]
        else:
            a11=''
        if ((0x0010,0x21A0) in ds):
            a12=ds[0x0010,0x21A0]
        else:
            a12=''
        if ((0x0010,0x21B0) in ds):
            a13=ds[0x0010,0x21B0]
        else:
            a13=''
        if ((0x0010,0x21D0) in ds):
            a14=ds[0x0010,0x21D0]
        else:
            a14=''
        if ((0x0010,0x2203) in ds):
            a15=ds[0x0010,0x2203]
        else:
            a15=''
        if ((0x0032,0x1066) in ds):
            a16=ds[0x0032,0x1066]
        else:
            a16=''
        if ((0x0032,0x1067) in ds):
            a17=ds[0x0032,0x1067]
        else:
            a17=''
        if ((0x0038,0x0010) in ds):
            a18=ds[0x0038,0x0010]
        else:
            a18=''
        if ((0x0038,0x0014) in ds):
            a19=ds[0x0038,0x0014]
        else:
            a19=''
        if ((0x0038,0x0060) in ds):
            a20=ds[0x0038,0x0060]
        else:
            a20=''
        if ((0x0038,0x0062) in ds):
            a21=ds[0x0038,0x0062]
        else:
            a21=''
        if ((0x0038,0x0064) in ds):
            a22=ds[0x0038,0x0064]
        else:
            a22=''
        if ((0x0038,0x0500) in ds):
            a23=ds[0x0038,0x0500]
        else:
            a23=''
            
        
        
        
        l.append([a,a1,a2,a3,a4,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,a20,a21,a22,a23]) 
        return l

            

        #col_names = ["Tag Name", "Tag Code","Tag Value"]

        #print(data)


   

    def getGeneralStudy(self,meta_data): ##Public func
        '''it will  returns  all the tags within getGeneralStudy  module
         Parameters:
                ds  :  scan meta deta

         Returns:
                List: Tag Name, Tag Code and Tag Values with in getGeneralStudy Module 
                        
        '''   
        l=[]
        #a = ds[0x0020,0x000e].value
        if ((0x0008,0x0020) in ds):
            a=ds[0x0008,0x0020]
        else:
            a=''
        if ((0x0008,0x0030) in ds):
            a1=ds[0x0008,0x0030]
        else:
            a1=''
        if ((0x0008,0x0050) in ds):
            a2=ds[0x0008,0x0050]
        else:
            a2=''
        if ((0x0008,0x0051) in ds):
            a3=ds[0x0008,0x0051]
        else:
            a3=''
        if ((0x0008,0x0090) in ds):
            a4=ds[0x0008,0x0090]
        else:
            a4=''
        if ((0x0008,0x0096) in ds):
            a5=ds[0x0008,0x0096]
        else:
            a5=''
        if ((0x0008,0x009C) in ds):
            a6=ds[0x0010,0x009C]
        else:
            a6=''
        if ((0x0008,0x009D) in ds):
            a7=ds[0x0010,0x009D]
        else:
            a7=''
        if ((0x0008,0x1030) in ds):
            a8=ds[0x0008,0x1030]
        else:
            a8=''
        if ((0x0008,0x1032) in ds):
            a9=ds[0x0008,0x1032]
        else:
            a9=''
        if ((0x0008,0x1048) in ds):
            a10=ds[0x0008,0x1048]
        else:
            a10=''
        if ((0x0008,0x1049) in ds):
            a11=ds[0x0008,0x1049]
        else:
            a11=''
        if ((0x0008,0x1060) in ds):
            a12=ds[0x0008,0x1060]
        else:
            a12=''
        if ((0x0008,0x1110) in ds):
            a13=ds[0x0008,0x1110]
        else:
            a13=''
        if ((0x0020,0x000D) in ds):
            a14=ds[0x0020,0x000D]
        else:
            a14=''
        if ((0x0020,0x0010) in ds):
            a15=ds[0x0020,0x0010]
        else:
            a15=''
        if ((0x0032,0x1033) in ds):
            a16=ds[0x0032,0x1033]
        else:
            a16=''
        if ((0x0032,0x01034) in ds):
            a17=ds[0x0032,0x01034]
        else:
            a17=''
        if ((0x0040,0x1012) in ds):
            a18=ds[0x0040,0x1012]
        else:
            a18=''
        
            
        
        
        
        l.append([a,a1,a2,a3,a4,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18]) 
        return l


    def getImagePixel(self,ds):    ##Public func
        '''it will  returns  all the tags within getImagePixel  module
         Parameters:
                ds  :  scan meta deta

         Returns:
                list : Tag Name, Tag Code and Tag Values with in getImagePixel Module 
                        
        '''
          
        l=[]
        #a = ds[0x0020,0x000e].value
        if ((0x0028,0x0002) in ds):
            a=ds[0x0028,0x0002]
        else:
            a=''
        if ((0x0028,0x0004) in ds):
            a1=ds[0x0028,0x0004]
        else:
            a1=''
        if ((0x0028,0x0006) in ds):
            a2=ds[0x0028,0x0006]
        else:
            a2=''
        if ((0x0028,0x0010) in ds):
            a3=ds[0x0028,0x0010]
        else:
            a3=''
        if ((0x0028,0x0010) in ds):
            a4=ds[0x0028,0x0010]
        else:
            a4=''
        if ((0x0028,0x0011) in ds):
            a5=ds[0x0028,0x0011]
        else:
            a5=''
        if ((0x0028,0x0034) in ds):
            a6=ds[0x0028,0x0034]
        else:
            a6=''
        if ((0x0028,0x0100) in ds):
            a7=ds[0x0028,0x0100]
        else:
            a7=''
        if ((0x0028,0x0101) in ds):
            a8=ds[0x0028,0x0101]
        else:
            a8=''
        if ((0x0028,0x0102) in ds):
            a9=ds[0x0028,0x0102]
        else:
            a9=''
        if ((0x0028,0x0103) in ds):
            a10=ds[0x0028,0x0103]
        else:
            a10=''
        if ((0x0028,0x0106) in ds):
            a11=ds[0x0028,0x0106]
        else:
            a11=''
        if ((0x0028,0x0107) in ds):
            a12=ds[0x0028,0x0107]
        else:
            a12=''
        if ((0x0028,0x0121) in ds):
            a13=ds[0x0008,0x0121]
        else:
            a13=''
        if ((0x0028,0x1101) in ds):
            a14=ds[0x0028,0x01101]
        else:
            a14=''
        if ((0x0028,0x1102) in ds):
            a15=ds[0x0028,0x1102]
        else:
            a15=''
        if ((0x0028,0x1103) in ds):
            a16=ds[0x0028,0x1103]
        else:
            a16=''
        if ((0x0028,0x1201) in ds):
            a17=ds[0x0028,0x1201]
        else:
            a17=''
        if ((0x0028,0x1202) in ds):
            a18=ds[0x0028,0x1202]
        else:
            a18=''
        if ((0x0028,0x1203) in ds):
            a19=ds[0x0028,0x1203]
        else:
            a19=''
        if ((0x0028,0x2000) in ds):
            a20=ds[0x0028,0x2000]
        else:
            a20=''
        if ((0x0028,0x2002) in ds):
            a21=ds[0x0028,0x2002]
        else:
            a21=''
        if ((0x0028,0x7FE0) in ds):
            a22=ds[0x0028,0x7FE0]
        else:
            a22=''
        if ((0x0028,0x7FE0) in ds):
            a23=ds[0x0028,0x7FE0]
        else:
            a23=''
        if ((0x7FE0,0x0001) in ds):
            a24=ds[0x7FE0,0x0001]
        else:
            a24=''
        if ((0x7FE0,0x0002) in ds):
            a25=ds[0x7FE0,0x0002]
        else:
            a25=''
        if ((0x7FE0,0x0010) in ds):
            a26=ds[0x7FE0,0x0010]
        else:
            a26=''
        
        l.append([a,a1,a2,a3,a4,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,a20,a21,a22,a23,a24,a25,a26]) 
        return l

In [ ]:
class modify:
    """
    A class to represent modify.

    ...

    Attributes
    ----------
    name : None
    

    Methods
    -------
    
    tag(tags,ds,value):
        it will  modify   tags  values in scan meta data
         
    image(ds,image):
        This function saves the provided image (rgb or gray scale) in the dataset
       


    maxDiameter(full_mask,spacing):
        This function finds the max diameter of 3D object
         
    """

    def __init__(self):
        
        pass
        """
        Constructs all the necessary attributes for the diplay object.

        Parameters
        ----------
            name : None
            
        """


    def tag(self,tags,ds,value):
        '''it will  modify   tags  values in scan meta data
         Parameters:
                ds  :  scan meta deta
                tags : tag number to modify
                value : new value for the tag

        Returns:
                ds : Meta data with new tag value 
                        
        '''        
        (x1,x2) = tags
        if (x1,x2) in ds:
            print('tag name: ',ds[x1,x2].keyword)
            print('previous value: ',ds[x1,x2].value)
            ds[x1,x2].value = value
            print('new value: ',ds[x1,x2].value)
        else:
            print('Tag not found')
        return ds

    def image(self,image,output_path,dummy_dcm_path=''):
        '''
        This function saves the provided image (rgb or gray scale) in the dataset
        Parameters:
                ds  :  scan meta deta
                image : numpy array
                

        Returns:
                ds : Meta data with image value 
                        
        '''
        if dummy_dcm_path=='':
            dummy_dcm_path = 'dummy.dcm'
        
        ds = pydicom.dcmread(dummy_dcm_path)
        
        ds.Rows = image.shape[0]
        ds.Columns = image.shape[1]

        if len(image.shape)>2: ##rgb
            image = image.astype(np.uint8)
            ds.PhotometricInterpretation = 'RGB'
            ds.PixelRepresentation = 0
            ds.WindowCenter = 127.5
            ds.WindowWidth = 255.0
            ds.SamplesPerPixel = 3
            ds.BitsAllocated = 8
            ds.BitsStored = 8
            ds.HighBit = 7
            if (0x0028, 0x0006) in ds:
                ds[0x0028, 0x0006].vaue = 0
            else:
                ds.add_new(0x00280006, 'US', 0)
        else:
            image = image.astype(np.int16)
        if (0x7fe0, 0x0010) in ds:
            ds.PixelData = image.tobytes()
        else:
            ds.add_new([0x7fe0, 0x0010], 'OB', image.tobytes())
        ds.save_as(output_path)



    def maxDiameter(full_mask,spacing):
        
        '''
        This function finds the max diameter of 3D object
         Parameters:
                full_mask  : it should contain all the slices of nodule
                spacing : pixel spacing
                

         Returns:
                ds : Meta data with image value 
                        
        '''
        # Ful mask means that it should contain all the slices of nodule, so shape can be like [6,45,45]
        # spacing is array contains spacing of x,y and z axis, shape should be like [0.7,1.5]

    #     full_mask = edge_mask(full_mask);    
        all_points = np.where(full_mask>0)
        x_coor = all_points[1]
        y_coor = all_points[2]
        z_coor = all_points[0]

        pairs = []
        coordinates = []
    #     print('Number of points is {}'.format(z_coor.min()))

        for i in range(z_coor.shape[0]):
            coor = [int(x_coor[i]),int(y_coor[i]),int(z_coor[i])]
            coordinates.append(coor)

        cpy = coordinates[:]

        for p in coordinates:
            cpy.remove(p)
            for points in cpy:
                pr = [p,points]
                pairs.append(pr)

        distances = np.zeros(len(pairs))
        i = 0
    #     print('Number of pairs is {}'.format(len(pairs)))
        for pair in pairs:
            p_1 = pair[0]
            p_2 = pair[1]
            distances[i] = math.sqrt(((p_1[0] - p_2[0])*spacing[0])**2 + ((p_1[1] - p_2[1])*spacing[0])**2 + ((p_1[2] - p_2[2])*spacing[1])**2)
            i = i+1
        max_dis = float(distances.max())
        max_p = np.where(distances==distances.max())[0]
        max_pints = []
        for p in max_p:
            max_pints.append(pairs[p])
        return max_dis, max_pints

In [ ]:
class process:
    """
    A class to represent process.

    ...

    Attributes
    ----------
    name : None
    

    Methods
    -------
    
    volumeCalculator(slices,spacing):
        Returns the volume of an object in 3D binary mask in physical measurements.
    
    getNormalized(scan,wc ,wl):
        Returns the normalized volume.

    windowing(self, scan,window_mode, window):
        apply window function on the scan
    """



    def __init__(self):
        """
        Constructs all the necessary attributes for the diplay object.

        Parameters
        ----------
            name : None
            
        """

        self.windowing = windowing()
    
    def volumeCalculator(self,slices,spacing):
        """
        Returns the volume of an object in 3D binary mask in physical measurements.

        This function takes 3D binary mask, of an object as an input along with pixel spacing and returns the physical volume as an output.     The unit of volume returned is same as the unit of pixel spacing (which is usually mm).

        :param slices: Slices containing binary mask, usually of tumor. 
        :type slices: int
        :param spacing: Takes in a list of pixel spacing as [px,py,pz] where px,py and pz are pixel spacings in x,y and z cordinates. 
        :type spacing: list

        :return: Returns the physical volume
        :rtype: float
        """
        #def __init__(self, spacing=[None,None,None]):
        self.spacing[0]=spacing[0]
        self.spacing[1]=spacing[1]
        self.spacing[2]=spacing[2]


        v=0

        if len(slices.shape) == 2:
            assert len(self.spacing) != 2, "Spacing should have two values, spacing should be a list of two numbers in case of 2D image"

            assert self.spacing[0] != None, "px cannot be None"
            assert self.spacing[1] != None, "py cannot be None"

            v = np.sum(slices)*self.spacing[0]*self.spacing[1]
        else:
            assert len(self.spacing) == 3, "pz should be defined"

            assert self.spacing[0] != None, "px cannot be None"
            assert self.spacing[1] != None, "py cannot be None"
            assert self.spacing[2] != None, "pz cannot be None"
            for i in range(slices.shape[-1]):
                v=np.sum(slices)*self.spacing[2]*self.spacing[0]*self.spacing[1]

        return v
    

    def getNormalized(self,scan,wc ,wl):
        """
        Returns the normalized volume.

        :param scan: 3D image to normalize. 
        :type nscan: int/float
        """
        mn = wc - wl
        mx = wc+wl
        d = mx - mn
        scan  = np.where(scan<mn,mn, scan)
        scan  = np.where(scan>mx,mx, scan)
        scan = (scan-mn)/d
        return scan 

    def resize3D(self, nscan,shape):
        """
        Returns the 3D volume after resizing it to the required shape.

        :param nscan: 3D image to resize. 
        :type nscan: int/float
        """
        
        from scipy import ndimage
        assert len(shape)==3,"The new shape should have 3 values"
        assert len(shape)==3, "Image should have 3 dimensions, given "+str(len(nscan.shape))+ " dimensions"
        depth_factor=shape[0]/nscan.shape[0]
        width_factor=shape[2]/nscan.shape[2]
        height_factor=shape[1]/nscan.shape[1]

        if mask==False:
            nscan = ndimage.zoom(nscan, (depth_factor, height_factor, width_factor), order=1)
        elif is_mask==True:
            nscan = ndimage.zoom(nscan, (depth_factor, height_factor, width_factor), order=1, mode = 'nearest' )
    
    
        return nscan
        

class windowing:
    
    def __init__(self):
        pass
        
    def manual(self, scan,window_mode, window):
        """
        Apply windowing/grey-level mapping on the scan.
        
        Parameters: input scan to be performed windowing on.
        type number: pydicom.dataset.FileDataset
    
        :return: The result of windowing.
        :rtype: numpy.ndarray
        """
        assert window_mode=='min/max' or window_mode=='wl/ww' or window_mode=='scan_min/scan_max', "window_mode should be defined!"
        assert len(window)==2, "Please define min/max or ww/wl values for windowing!"
    
        import numpy as np
        if window_mode=='wl/ww':
            assert window[0]!=None and self.window[1]!=None, "WW/WL values cannot be None!" 
            wl=self.window[0]
            ww=self.window[1]
            mn = wl - ww/2
            mx = wl + ww/2
        elif self.window_mode=='min/max':
            assert self.window[0]!=None and self.window[1]!=None, "Min/max values cannot be None!"
            mn=self.window[0]
            mx=self.window[1]
        elif self.window_mode=='scan_min/scan_max':
            mn = np.amin(scan)
            mx = np.amax(scan)
        
        d = mx - mn
        scan  = np.where(scan<mn,mn, scan)
        scan  = np.where(scan>mx,mx, scan)

        return scan
    
    def manual(self, scan,ds):
        """
        apply window function on the scan
        
        :param scan: input scan to be performed windowing on.
        :type number: pydicom.dataset.FileDataset
    
        :return: The result of windowing.
        :rtype: numpy.ndarray
        """
        #assert window_mode=='min/max' or window_mode=='wl/ww' or window_mode=='scan_min/scan_max', "window_mode should be defined!"
        #assert len(window)==2, "Please define min/max or ww/wl values for windowing!"
        
        wc,ww,intercept,slope = data[('0028','1050')].value,data[('0028','1051')].value, data[('0028','1052')].value, data[('0028','1053')].value 
        mn = wc - ww/2
        mx = wc + ww/2

        scan  = np.where(scan<mn,mn, scan)
        scan  = np.where(scan>mx,mx, scan)
        return scan
    
    def seg1(self,scan):
        def normal(un):
            mn = np.amin(un)
            mx = np.amax(un)
            d = mx - mn
            normalized = (un-mn)/d
            return normalized

        def get_normalized(scan):
            wc= (300-(-1000))/(1000-(-1000)) #normalizing WC
            w= (1000-(-1000))/(1000-(-1000))  #normalizing WW
            wl=w/2
            mn = wc - wl
            mx = wc+wl
            d = mx - mn
            scan  = np.where(scan<mn,mn, scan)
            scan  = np.where(scan>mx,mx, scan)
            scan=normal(scan)
            return scan        
        
        final_image=get_normalized(scan)

        return final_image
    
    def generic_window(self,ds):
        
        def get_normalized(scan,wc ,wl):
            mn = wc - wl
            mx = wc+wl
            d = mx - mn
            scan  = np.where(scan<mn,mn, scan)
            scan  = np.where(scan>mx,mx, scan)
            scan = (scan-mn)/d
            return scan  

        wc= ds[0x28,0x1050].value  #window center
        w=(ds[0x28,0x1051].value) #window width
        ww=int(w/2)

        new_wc=wc+(0.3*wc)
        new_ww=(0.8*ww)   

        img=ds.pixel_array
        result=get_normalized(img,wc+(0.3*wc),(0.8*ww) )

        return result


    def segmentBones(self,scan,ds):
        
        try: 
            wc= ds[0x28,0x1050].value  #window center
            w=ds[0x28,0x1051].value #window width
            res = self.generic_window(ds)
        except:
            res = self.seg1(scan)
        
        return res

In [ ]:
class display:
    """ 
    ImageSliceViewer3D is for viewing volumetric image slices in jupyter or
    ipython notebooks. 
    
    User can interactively change the slice plane selection for the image and 
    the slice plane being viewed. 

    Argumentss:
    Volume = 3D input image
    figsize = default(8,8), to set the size of the figure
    cmap = default('gray'), string for the matplotlib colormap. You can find 
    more matplotlib colormaps on the following link:
    https://matplotlib.org/users/colormaps.html
    
    """
    
    def __init__(self):
        pass
    
        #ipyw.interact(self.multiSliceViewer)
        
    def multiSliceViewer(self,volume ,figsize=(100,100), cmap='gray'):
        self.volume = volume
        self.figsize = figsize
        self.cmap = cmap
        self.v = [np.min(volume), np.max(volume)]
        ipyw.interact(self.multiSlice)
        
    
    def multiSlice(self):

        #ipyw.interact(self.views)
        self.vol1 = np.transpose(self.volume, [1,2,0])
        self.vol2 = np.rot90(np.transpose(self.volume, [2,0,1]), 3) #rotate 270 degrees
        self.vol3 = np.transpose(self.volume, [0,1,2])
        maxZ1 = self.vol1.shape[2] - 1
        maxZ2 = self.vol2.shape[2] - 1
        maxZ3 = self.vol3.shape[2] - 1
        ipyw.interact(self.plot_slice, 
            z1=ipyw.IntSlider(min=0, max=maxZ1, step=1, continuous_update=False, 
            description='Axial:'), 
            z2=ipyw.IntSlider(min=0, max=maxZ2, step=1, continuous_update=False, 
            description='Coronal:'),
            z3=ipyw.IntSlider(min=0, max=maxZ3, step=1, continuous_update=False, 
            description='Sagittal:'))
        
    def plot_slice(self, z1, z2, z3):
        # Plot slice for the given plane and slice
        f,ax = plt.subplots(1,3, figsize=self.figsize)
        #print(self.figsize)
        #self.fig = plt.figure(figsize=self.figsize)
        #f(figsize = self.figsize)
        ax[0].imshow(self.vol1[:,:,z1], cmap=plt.get_cmap(self.cmap), 
            vmin=self.v[0], vmax=self.v[1])
        ax[1].imshow(self.vol2[:,:,z2], cmap=plt.get_cmap(self.cmap), 
            vmin=self.v[0], vmax=self.v[1])
        ax[2].imshow(self.vol3[:,:,z3], cmap=plt.get_cmap(self.cmap), 
            vmin=self.v[0], vmax=self.v[1])
        plt.show()


    def plotSlice(self, image):
        '''
        Displays the 2D plot of an image.
        
        Parameters:
                image (int)(float): A 2d numpy array

        Returns:
                Displays an Image.  
        '''        
        plt.imshow(image,'gray')
        

    def display3D(self,volume):
        '''Displays the 3D plot given a mask
        
        Parameters:
                image (int)(float): A 3d numpy array

        Returns:
                Plot 3d Image  
        '''

        p=volume.transpose(2,1,0)
        p = p[:,:,::-1]
        threshold=0.5
        alpha=0.5
        verts, faces, norm, val = measure.marching_cubes_lewiner(p, threshold)

        fig = plt.figure(figsize=(20, 20))
        ax = fig.add_subplot(111, projection='3d')

        # Fancy indexing: `verts[faces]` to generate a collection of triangles
        mesh = Poly3DCollection(verts[faces], alpha=alpha)
        face_color = [0.8, 0.2, 1]
        mesh.set_facecolor(face_color)
        ax.add_collection3d(mesh)

        ax.set_xlim(0, p.shape[0])
        ax.set_ylim(0, p.shape[1])
        ax.set_zlim(0, p.shape[2])

        plt.show()

    def intensityHist(self,array):
        
        '''
        it will plot intensity histogram of given array        
        Parameters:
                image (int)(float): A 2d or 3d numpy array

        Returns:
                Plots Image histogram  
        '''
        array = np.array(array)

        [counts,bins,bars]=plt.hist(array.flatten())     
        